In [1]:
# Install default libraries
import os
import pathlib
import sys
import urllib

# Import installed modules
import imageio
import numpy as  np
import pandas as pd
import tqdm

In [2]:
# Now we need to define a main window class that is needed to set a window size / resolution.
# Based on the QT4 example:
# https://github.com/almarklein/visvis/blob/master/examples/embeddingInQt4.py
from PyQt5.QtWidgets import QWidget, QHBoxLayout

# Import visvis
import visvis as vv

# Define the class
class MainWindow(QWidget):
    def __init__(self, *args):
        QWidget.__init__(self, *args)
        self.fig = vv.backends.backend_pyqt5.Figure(self)
        self.sizer = QHBoxLayout(self)
        self.sizer.addWidget(self.fig._widget)
        self.setLayout(self.sizer)
        self.setWindowTitle("Rosetta")
        self.show()

def download_file(dl_path, dl_url):
    """
    download_file(dl_path, dl_url)

    This helper function supports one to download files from the Internet and
    stores them in a local directory.

    Parameters
    ----------
    dl_path : str
        Download path on the local machine, relative to this function.
    dl_url : str
        Download url of the requested file.
    """

    # Obtain the file name from the url string. The url is split at
    # the "/", thus the very last entry of the resulting list is the file's
    # name
    file_name = dl_url.split('/')[-1]

    # Create necessary sub-directories in the DL_PATH direction (if not
    # existing)
    pathlib.Path(dl_path).mkdir(parents=True, exist_ok=True)

    # If the file is not present in the download directory -> download it
    if not os.path.isfile(dl_path + file_name):

        # Download the file with the urllib  package
        urllib.request.urlretrieve(dl_url, dl_path + file_name)

In [3]:
# Set a local download path
dl_path = "shape_models/"

# Set dictionary with 2 different resolutions
shape_models = \
    {"67P" : "https://naif.jpl.nasa.gov/pub/naif/ROSETTA/kernels/dsk/ROS_CG_M004_OSPGDLR_N_V1.OBJ"}

# Which model?
model = "67P"

# If file not present: download it!
if not pathlib.Path(f"{dl_path + shape_models[model].split('/')[-1]}").is_file():
    
    # Download the shape model, create (if needed) the download path and store the data set
    download_file(dl_path, shape_models[model])

In [4]:
# Load the shape model
shape_obj = pd.read_csv(f"{dl_path + shape_models[model].split('/')[-1]}", \
                        delim_whitespace=True, \
                        names=["TYPE", "X1", "X2", "X3"])

# Assign the vertices and faces
vertices = shape_obj.loc[shape_obj["TYPE"] == "v"][["X1", "X2", "X3"]].values.tolist()
faces = shape_obj.loc[shape_obj["TYPE"] == "f"][["X1", "X2", "X3"]].values

# The index in the faces sub set starts at 1. For Python, it needs to start at 0.
faces = faces - 1

# Convert the indices to integer
faces = faces.astype(int)

# Convert the numpy array to a Python list
faces = faces.tolist()

In [5]:
# Now let"s create an animation

# Create visvis application
app = vv.use()
app.Create()

# Create main window frame and set a resolution.
main_w = MainWindow()
main_w.resize(500, 500)

# Create the 3 D shape model as a mesh. verticesPerFace equals 3 since triangles define the
# mesh"s surface in this case
shape_obj = vv.mesh(vertices=vertices, faces=faces, verticesPerFace=3)
shape_obj.specular = 0.0
shape_obj.diffuse = 0.9

# Get figure
figure = vv.gcf()

# Get axes objects and set figure parameters
axes = vv.gca()
axes.bgcolor = (0, 0, 0)
axes.axis.showGrid = False
axes.axis.visible = False

# Set camera settings
#
axes.camera = "3d"
axes.camera.fov = 60
axes.camera.zoom = 0.1

# Turn off the main light
axes.light0.Off()

# Create a fixed light source
light_obj = axes.lights[1]
light_obj.On()
light_obj.position = (5.0, 5.0, 5.0, 0.0)

# Empty array that contains all images of the comet"s rotation
comet_images = []

# Rotate camera in 300 steps in azimuth
for azm_angle in tqdm.tqdm(range(360)):

    # Change azimuth angle of the camera
    axes.camera.azimuth = float(azm_angle)

    # Draw the axes and figure
    axes.Draw()
    figure.DrawNow()

    # Get the current image
    temp_image = vv.getframe(vv.gca())

    # Apped the current image in 8 bit integer
    comet_images.append((temp_image*255).astype(np.uint8))

# Save the images as an animated GIF
imageio.mimsave(f"gifs/{shape_models[model].split('/')[-1]}.gif", comet_images, duration=0.04)

100%|█████████████████████████████████████████████████████████████| 360/360 [00:26<00:00, 13.52it/s]
